In [ ]:
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import requests
from Bio import SeqIO
import os

# Función para obtener secuencia proteica desde UniProt
def obtener_proteina_uniprot(url):
    response = requests.get(url)
    if response.status_code == 200:
        fasta = response.text
        secuencia = ""
        for linea in fasta.splitlines():
            if not linea.startswith(">"):
                secuencia += linea.strip()
        return secuencia
    else:
        raise Exception(f"Error al descargar: {url}")

# Obtener genes de resistencia desde UniProt
blatem1_prot = obtener_proteina_uniprot("https://rest.uniprot.org/uniprotkb/Q6SJ61.fasta")
blactxm15_prot = obtener_proteina_uniprot("https://rest.uniprot.org/uniprotkb/H6UQH2.fasta")

genes_proteicos = {
    "blaTEM-1": blatem1_prot,
    "blaCTX-M-15": blactxm15_prot
}

# Función para analizar un archivo .fna
def buscar_gen_por_proteina(fna_path, genes_proteicos):
    resultados = {nombre: [] for nombre in genes_proteicos}
    for record in SeqIO.parse(fna_path, "fasta"):
        adn = record.seq
        for strand in [adn, adn.reverse_complement()]:
            for frame in range(3):
                proteina = strand[frame:].translate(to_stop=False)
                proteina_str = str(proteina)
                for nombre_gen, secuencia_prot in genes_proteicos.items():
                    if secuencia_prot in proteina_str:
                        resultados[nombre_gen].append(record.id)
                        break
    return resultados

# Lista de archivos .fna a analizar
archivos_fna = [
    "GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna",
    "GCF_050516345.1_ASM5051634v1_genomic.fna",
    "GCF_050516355.1_ASM5051635v1_genomic.fna",
    "GCF_050516365.1_ASM5051636v1_genomic.fna",
    "GCF_050516375.1_ASM5051637v1_genomic.fna",
    "GCF_050516385.1_ASM5051638v1_genomic.fna",
    "GCF_050563505.1_ASM5056350v1_genomic.fna",
    "GCF_050568845.1_2848_genomic.fna"
]

# Función para crear documento Word
def crear_documento(archivo_fna, resultados):
    # Extraer nombre base del archivo
    nombre_base = os.path.splitext(archivo_fna)[0]
    
    # Crear documento Word
    doc = Document()

    # Definir estilo base
    style = doc.styles['Normal']
    font = style.font
    font.name = 'Tahoma'
    font.size = Pt(10)

    def add_paragraph(text, bold=False, alignment='left', spacing_after=0, numbered=False, number=None):
        p = doc.add_paragraph()
        if numbered and number is not None:
            run = p.add_run(f"{number}. {text}")
        else:
            run = p.add_run(text)
        run.bold = bold
        if alignment == 'center':
            p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        elif alignment == 'right':
            p.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
        elif alignment == 'justify':
            p.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
        else:
            p.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT
        p.paragraph_format.space_after = Pt(spacing_after)
        return p

    # Encabezado
    add_paragraph("MODELO DE INFORME DE CORRELACIÓN\nCLÍNICA ADMINISTRATIVA DEL SUS", bold=True, alignment='center', spacing_after=10)

    # Datos de cabecera
    add_paragraph("A:\t\tDIRECTOR MUNICIPAL DE SALUD GAMLP\n\t\t\tGOBIERNO AUTÓNOMO MUNICIPAL DE LA PAZ\n", spacing_after=10)
    add_paragraph("VÍA:\t\tLABORATORIO FARMACOLÓGICO\n", spacing_after=10)
    add_paragraph("DE:\t\tRESPONSABLE DE LA ELABORACIÓN DEL INFORME\n\t\t\tBIOINGENIERO BALDRIC CARDENAS\n", spacing_after=10)
    add_paragraph("REF.:\tINFORME DE REVISIÓN DEL CUMPLIMIENTO DE INFORME GENÉTICO DE LA BACTERIA E. COLI \"blatem1\" y \"blactxm15\" REALIZADA EN EL HOSPITAL MUNICIPAL DE SEGUNDO NIVEL LA PORTADA CORRESPONDIENTE AL MESE DE JUNIO DE LA GESTION 2025\n", spacing_after=10)
    add_paragraph("Fecha:\t\t09 de junio de 2025\n", spacing_after=10)

    # Línea de presentación
    add_paragraph("De mi mayor consideración:\n", bold=True, spacing_after=10)

    # Cuerpo del documento
    add_paragraph(
        "En cumplimiento a lo establecido en el Reglamento para la aplicación técnica y la gestión administrativa y financiera de la Ley N° 1152, aprobado mediante Resolución Ministerial N° 0251 de 30 de junio de 2021, remito el informe de revisión del cumplimiento de la Correlación Clínica Administrativa de los Servicios de Salud otorgados a la población beneficiaria, en el Hospital Municipal de Segundo Nivel La Portada la gestión 2025",
        alignment='justify', spacing_after=10
    )

    # Secciones
    secciones = {
        "ANTECEDENTES": [
            "La Ley N° 475, de 30 de diciembre de 2013, de Prestaciones de Servicios de Salud Integral del Estado Plurinacional de Bolivia, tiene por objeto establecer y regular la atención integral y la protección financiera en salud de la población beneficiaria que no se encuentre cubierta por el Seguro Social Obligatorio de Corto Plazo y establecer las bases para la universalización de la atención integral en salud.",
            "La Ley N° 1069, de 28 de mayo de 2018, modifica la Ley N° 475, para optimizar el uso de los recursos financieros asignados a la atención integral de salud.",
            "La Ley N° 1152, de 20 de febrero de 2019, modifica la Ley N° 475, para ampliar la población beneficiaria que no se encuentra cubierta por la Seguridad Social de Corto Plazo, con atención gratuita en salud, en avance hacia un Sistema Único de Salud, Universal y Gratuito.",
            "La Resolución Ministerial N° 0251, de 30 de junio de 2021, aprueba el Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de La Ley N° 1152 de 20 de febrero de 2019 \"Hacia el Sistema Único de Salud Universal y Gratuito\"."
        ],
        "JUSTIFICACIÓN": [
            "La correlación clínica administrativa, es la coherencia existente entre los diagnósticos y procedimientos efectuados por el personal de salud, registrados en el expediente clínico, que comprende la historia clínica, epicrisis, hojas de enfermería, hojas de evolución y otros pertinentes a la atención, con los servicios de salud declarados en los documentos administrativos generados por las herramientas informáticas del establecimiento, la misma que debe ser realizada con el propósito de mejorar la calidad del dato y la atención de los pacientes y retroalimentar al personal de salud y administrativo.",
            "El Hospital La Portada, en cumplimiento al Parágrafo VI, inciso b) del Artículo 53 del Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152, que señala como uno de los documentos para el proceso de cobro intermunicipal a los GAM's o GAIOC's deudores, el informe de Correlación Clínica Administrativa de los pacientes referidos y en tránsito de dichos municipios."
        ],
        "METODOLOGIA": [
            "El Informe de Correlación Clínica Administrativa, se elaboró bajo la siguiente metodología:",
            "a) Se realizó la revisión de los expedientes clínicos de pacientes atendidos según incisos a) y b) del parágrafo I del artículo 53 del Reglamento para la Aplicación Técnica y la Gestión Administrativa y Financiera de la Ley N° 1152, incluidos en los cobros intermunicipales y se los comparó con los registros del SIAF del establecimiento.",
            "b) Para este fin se aplicó el instrumento de verificación proporcionado por el Ministerio de Salud y Deportes, que incluye las siguientes variables:\n· Verificación de los datos generales de cada paciente.\n· Identificación del Diagnóstico Principal.\n· Identificación de posibles complicaciones y comorbilidades del paciente.\n· Procedimientos terapéuticos y diagnósticos otorgados.",
            "c) Identificación y descripción de las principales inconsistencias encontradas.",
            "d) Descripción de las acciones correctivas realizadas en el establecimiento de salud."
        ],
        "ANÁLISIS TÉCNICO": [],
        "RESULTADOS": [
            "De la revisión de la muestra señalada de los expedientes clínicos y de los servicios de salud declarados en el SIAF del establecimiento de salud, se obtuvieron los siguientes resultados generales:",
            "Acciones correctivas realizadas",
            "Con los resultados de la revisión de la correlación clínica administrativa del hospital de Segundo Nivel La Portada, se realizaron las siguientes acciones correctivas por parte de la Dirección del establecimiento de salud, con insumos proporcionados por el responsable del SUS del Hospital:",
            "1. Se retirará del cobro intermunicipal al GAM o GAIOC deudor, aquellos expedientes clínicos que presentan observaciones."
        ],
        "CONCLUSIONES": [
            f"1. Se realizó la verificación de la posible resistencia de la bacteria E. coli en el archivo {archivo_fna}. De la que se sospecha posee el gen de resistencia \"blatem1\", del análisis genético realizado se puede concluir que esta es:\n\n{'POSITIVO' if resultados['blaTEM-1'] else 'NEGATIVO'}\n",
            f"2. Se realizó la verificación de la posible resistencia de la bacteria E. coli en el archivo {archivo_fna}. De la que se sospecha posee el gen de resistencia \"blactxm15\", del análisis genético realizado se puede concluir que esta es:\n\n{'POSITIVO' if resultados['blaCTX-M-15'] else 'NEGATIVO'}\n"
        ]
    }

    # Escribir secciones
    for titulo, parrafos in secciones.items():
        numbered_sections = ["ANTECEDENTES", "JUSTIFICACIÓN", "METODOLOGIA", "ANÁLISIS TÉCNICO", "CONCLUSIONES"]
        is_numbered = titulo in numbered_sections
        # Agregar el título de la sección con número si corresponde
        number = numbered_sections.index(titulo) + 1 if is_numbered else None
        add_paragraph(titulo, bold=True, numbered=is_numbered, number=number, spacing_after=10)
        # Agregar los párrafos de la sección
        for i, parrafo in enumerate(parrafos, 1):
            # Para RESULTADOS y CONCLUSIONES, algunas subsecciones tienen numeración propia
            if titulo == "RESULTADOS" and i == 4:  # El último punto de RESULTADOS tiene "1."
                add_paragraph(parrafo, alignment='justify', numbered=True, number=1, spacing_after=10)
            elif titulo == "CONCLUSIONES":  # CONCLUSIONES ya tiene numeración en el texto
                add_paragraph(parrafo, alignment='justify', spacing_after=10)
            else:
                add_paragraph(parrafo, alignment='justify', spacing_after=10)

    # Firma
    add_paragraph("\nIngeniero Baldric Cardenas\nC.I.: 12483476\n\nCc. Archivo.", alignment='left', spacing_after=10)

    # Guardar documento
    output_path = f"informe_bacteria_{nombre_base}.docx"
    doc.save(output_path)
    print(f"Documento guardado en: {output_path}")

# Procesar cada archivo
for archivo in archivos_fna:
    print(f"\nAnalizando {archivo}...")
    try:
        resultados = buscar_gen_por_proteina(archivo, genes_proteicos)
        print(f"Resultados para {archivo}:")
        print(f"- blaTEM-1 encontrado en {len(resultados['blaTEM-1'])} secuencias")
        print(f"- blaCTX-M-15 encontrado en {len(resultados['blaCTX-M-15'])} secuencias")
        
        # Crear documento Word para este archivo
        crear_documento(archivo, resultados)
        
    except Exception as e:
        print(f"Error al procesar {archivo}: {str(e)}")


Analizando GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna...
Resultados para GCF_050389795.1_MS10495_ARL02201_ST410_genomic.fna:
- blaTEM-1 encontrado en 0 secuencias
- blaCTX-M-15 encontrado en 0 secuencias
Documento guardado en: informe_GCF_050389795.1_MS10495_ARL02201_ST410_genomic.docx

Analizando GCF_050516345.1_ASM5051634v1_genomic.fna...
Resultados para GCF_050516345.1_ASM5051634v1_genomic.fna:
- blaTEM-1 encontrado en 1 secuencias
- blaCTX-M-15 encontrado en 0 secuencias
Documento guardado en: informe_GCF_050516345.1_ASM5051634v1_genomic.docx

Analizando GCF_050516355.1_ASM5051635v1_genomic.fna...
Resultados para GCF_050516355.1_ASM5051635v1_genomic.fna:
- blaTEM-1 encontrado en 0 secuencias
- blaCTX-M-15 encontrado en 0 secuencias
Documento guardado en: informe_GCF_050516355.1_ASM5051635v1_genomic.docx

Analizando GCF_050516365.1_ASM5051636v1_genomic.fna...
Resultados para GCF_050516365.1_ASM5051636v1_genomic.fna:
- blaTEM-1 encontrado en 2 secuencias
- blaCTX-M-15 encont